# Prison Allocation 
This section will cover the prison allocation task only, we will not in this example include the early release functionality as proof that this model functions as exptected.  

First lets install the neccisary packages 

In [1]:
%pip install -r resources.txt

  Cloning https://github.com/conjure-cp/conjure-notebook.git (to revision v0.0.10) to /tmp/pip-req-build-c9zt6fdr
  Running command git clone --filter=blob:none --quiet https://github.com/conjure-cp/conjure-notebook.git /tmp/pip-req-build-c9zt6fdr
  Running command git checkout -q d58ac9af77e8c8b8d2c3e9633384fb3670fd03e5
  Resolved https://github.com/conjure-cp/conjure-notebook.git to commit d58ac9af77e8c8b8d2c3e9633384fb3670fd03e5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Note: you may need to restart the kernel to use updated packages.


# 1. Data Loading and preprocessing 
## 1.1 Context
Lets load in the prison template into this session. For context, the prison data contains an example of a prison system with capacity 270 and population 250 (15 empty beds). The 15 empty beds are distributed across different genders, supervision levels, and age categories:

**Adult High Supervision:**
- 2 Male beds: S1W1A-C08-B2, S1W1A-C08-B3
- 1 Female bed: S1W1B-C05-B3

**Adult Medium Supervision:**
- 2 Male beds: S1W2A-C07-B2, S1W2A-C07-B3
- 1 Female bed: S1W2B-C04-B4

**Adult Low Supervision:**
- 2 Male beds: S2W1A-C08-B1, S2W1A-C08-B3
- 1 Female bed: S2W1B-C05-B3

**Young Offenders High Supervision:**
- 1 Female bed: S3W1B-C05-B2

**Young Offenders Low Supervision:**
- 5 Male beds: S3W2A-C07-B3, S3W2A-C07-B4, S3W2C-C07-B4, S3W2C-C10-B4, S3W2C-C11-B4

## 1.2 Load and preprocess the data
In the code below we will load in and preprocess the data to get it to a suitable structure that can be read into the system. 

In [2]:
import json
import pandas as pd
import sys
sys.path.append('Preprocessing')
from PrisonDataPreprocessor import PrisonDataPreprocessor

# Initialize preprocessor
preprocessor = PrisonDataPreprocessor()

# Load and process prison template
df_beds = preprocessor.load_prison_template('Data/PrisonTemplate.json')

print(f"Total beds in dataset: {len(df_beds)}")
print(f"Occupied beds: {df_beds['occupied'].sum()}")
print(f"Empty beds: {(~df_beds['occupied']).sum()}")
print(f"\nDataFrame shape: {df_beds.shape}")
print(f"\nFirst few rows:")
df_beds.head()

Total beds in dataset: 270
Occupied beds: 255
Empty beds: 15

DataFrame shape: (270, 12)

First few rows:


,section_id,section_name,age_category,ward_id,supervision_level,wing_id,sex_assignment,cell_id,cell_type,bed_id,occupied,prisoner_id
0,S1,North Wing Section,Adult,S1W1,High,S1W1A,Male,S1W1A-C01,single,S1W1A-C01-B1,True,P00001
1,S1,North Wing Section,Adult,S1W1,High,S1W1A,Male,S1W1A-C02,single,S1W1A-C02-B1,True,P00002
2,S1,North Wing Section,Adult,S1W1,High,S1W1A,Male,S1W1A-C03,single,S1W1A-C03-B1,True,P00003
3,S1,North Wing Section,Adult,S1W1,High,S1W1A,Male,S1W1A-C04,single,S1W1A-C04-B1,True,P00004
4,S1,North Wing Section,Adult,S1W1,High,S1W1A,Male,S1W1A-C05,single,S1W1A-C05-B1,True,P00005


As we can see above the dataset should reflect what was mentioned in the context section. Outside of the previously mentioned, we can see that this data is in need for further preprocessing and domain reduction. Domain reduction is warranted in this case as we have very little use for data such as names or the cell type in our modelling process. 

Prior to doing any further data preprocessing, lets check we have the neccisary beds available to allocate. If there is not enough beds available when it is passed to the model, then we will see a "No Sollution" message as expected.

In [3]:
cell_spaces = df_beds[df_beds['occupied'] == 0]
print(f"\nTotal empty beds available for allocation: {len(cell_spaces)}")
print(f"\n List all available beds: ")
print(cell_spaces)


Total empty beds available for allocation: 15

 List all available beds: 
    section_id        section_name     age_category ward_id supervision_level  \
14          S1  North Wing Section            Adult    S1W1              High   
15          S1  North Wing Section            Adult    S1W1              High   
26          S1  North Wing Section            Adult    S1W1              High   
47          S1  North Wing Section            Adult    S1W2            Medium   
48          S1  North Wing Section            Adult    S1W2            Medium   
62          S1  North Wing Section            Adult    S1W2            Medium   
98          S2  South Wing Section            Adult    S2W1               Low   
100         S2  South Wing Section            Adult    S2W1               Low   
120         S2  South Wing Section            Adult    S2W1               Low   
176         S3   East Wing Section  Young Offenders    S3W1              High   
202         S3   East Wing Section

## 1.3 Encoding of Data 
As mentioned previously, the next step is to convert the data into a form interpretable to the model. This process will only convert those columns that are not currently of a numerical value. 



In [4]:
# Encode all non-numerical data
df_beds = preprocessor.encode_dataframe(df_beds)
print(f"\nDataFrame after encoding non-numerical data:")
print(df_beds.head())



DataFrame after encoding non-numerical data:
   section_id  section_name  age_category  ward_id  supervision_level  \
0         0.0           1.0           0.0      0.0                0.0   
1         0.0           1.0           0.0      0.0                0.0   
2         0.0           1.0           0.0      0.0                0.0   
3         0.0           1.0           0.0      0.0                0.0   
4         0.0           1.0           0.0      0.0                0.0   

   wing_id  sex_assignment  cell_id  cell_type  bed_id  occupied  prisoner_id  
0      0.0             1.0      0.0        1.0     0.0       1.0          0.0  
1      0.0             1.0      1.0        1.0     1.0       1.0          1.0  
2      0.0             1.0      2.0        1.0     2.0       1.0          2.0  
3      0.0             1.0      3.0        1.0     3.0       1.0          3.0  
4      0.0             1.0      4.0        1.0     4.0       1.0          4.0  


## 1.4 Domain Reduction
Beyond reducing unnecisary columns, we also remove cells that are already occupied, since including them in the model would be redudant - we will never be able to allocate anything to them. 

In [5]:

# Recreate cell_spaces after encoding
cell_spaces = df_beds[df_beds['occupied'] == 0].copy()
print(f"\nEmpty beds after encoding: {len(cell_spaces)}")


Empty beds after encoding: 15


## 1.5 Load in and preprocess new incoming prisoners
Using the encoding tool we used on the resident prisoner list, we now employ the same strategy of loading and encoding the data on the inbound prisoner data.

In [6]:
# Load incoming prisoners to be allocated
prisoners = preprocessor.load_prisoner_list('Data/AllocatedPrisonerList.json', 
                                           key='incoming_prisoners', 
                                           limit=15)

print(f"\nTotal prisoners to be allocated: {len(prisoners)}")
print("\nFirst few prisoners:")
for i, prisoner in enumerate(prisoners[:3]):
    print(prisoner)

# Encode incoming prisoners using the same encoder
prisoners_encoded = preprocessor.encode_prisoners(prisoners)

print(f"\nFirst prisoner after encoding:")
print(prisoners_encoded[0])


Total prisoners to be allocated: 15

First few prisoners:
{'prisoner_id': 'P00266', 'name': 'Angus "Mad Dog" MacDuff', 'sex': 'Male', 'age_category': 'Adult', 'supervision_level': 'High'}
{'prisoner_id': 'P00267', 'name': 'Bruce McLean', 'sex': 'Male', 'age_category': 'Adult', 'supervision_level': 'High'}
{'prisoner_id': 'P00268', 'name': 'Ailsa Morrison', 'sex': 'Female', 'age_category': 'Adult', 'supervision_level': 'High'}

First prisoner after encoding:
{'prisoner_id': 266, 'name': 'Angus "Mad Dog" MacDuff', 'sex': 'Male', 'age_category': 'Adult', 'supervision_level': 'High', 'age_category_encoded': 0, 'sex_encoded': 1, 'supervision_level_encoded': 0}


# 2. Running the Model

## 2.1 Preparation for feeding into the model 
Just before the model is run, we are requirerd to structure the data in an appropriate way for a successful model run. Below we establish the sets, domains to be used within the model. We also establish the criteria for the minimum time served which can be adjusted as needed. 

In [7]:
# Prepare data for Conjure constraint solver
mappings = preprocessor.prepare_allocation_mappings(prisoners_encoded, cell_spaces)

# Extract individual mappings for clarity
incoming_prisoners_sex = mappings['incoming_prisoners_sex']
incoming_prisoners_age = mappings['incoming_prisoners_age']
incoming_prisoners_supervision = mappings['incoming_prisoners_supervision']
prison_beds_sex = mappings['prison_beds_sex']
prison_beds_age = mappings['prison_beds_age']
prison_beds_supervision = mappings['prison_beds_supervision']
prisoner_min = mappings['prisoner_min']
prisoner_max = mappings['prisoner_max']
bed_min = mappings['bed_min']
bed_max = mappings['bed_max']

print(f"\nPrepared {len(prisoners_encoded)} incoming prisoners")
print(f"Prepared {bed_max} available bed spots")
print(f"\nPrisoner ID range: {prisoner_min} to {prisoner_max}")
print(f"Bed ID range: {bed_min} to {bed_max}")
print(f"\nIncoming Prisoners Sex: {incoming_prisoners_sex}")
print(f"\nIncoming Prisoners Age: {incoming_prisoners_age}")
print(f"\nIncoming Prisoners Supervision: {incoming_prisoners_supervision}")
print(f"\nPrison Bed Sex: {prison_beds_sex}")
print(f"\nPrison Bed Age: {prison_beds_age}")
print(f"\nPrison Bed Supervision: {prison_beds_supervision}")


Prepared 15 incoming prisoners
Prepared 15 available bed spots

Prisoner ID range: 266 to 280
Bed ID range: 1 to 15

Incoming Prisoners Sex: {266: 1, 267: 1, 268: 0, 269: 1, 270: 1, 271: 0, 272: 1, 273: 1, 274: 0, 275: 0, 276: 1, 277: 1, 278: 1, 279: 1, 280: 1}

Incoming Prisoners Age: {266: 0, 267: 0, 268: 0, 269: 0, 270: 0, 271: 0, 272: 0, 273: 0, 274: 0, 275: 1, 276: 1, 277: 1, 278: 1, 279: 1, 280: 1}

Incoming Prisoners Supervision: {266: 0, 267: 0, 268: 0, 269: 2, 270: 2, 271: 2, 272: 1, 273: 1, 274: 1, 275: 0, 276: 1, 277: 1, 278: 1, 279: 1, 280: 1}

Prison Bed Sex: {1: 1, 2: 1, 3: 0, 4: 1, 5: 1, 6: 0, 7: 1, 8: 1, 9: 0, 10: 0, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1}

Prison Bed Age: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1}

Prison Bed Supervision: {1: 0, 2: 0, 3: 0, 4: 2, 5: 2, 6: 2, 7: 1, 8: 1, 9: 1, 10: 0, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1}


## 2.2 Run the Model
Now that all the data is ready to be fed into the model, we can now write our model and run it to get our cell allocations

In [8]:
%load_ext conjure 

<IPython.core.display.Javascript object>

Conjure extension is loaded - run `%conjure_help`

In [9]:
%%conjure --solver=z3

$ Define ranges for prisoners and beds
given prisoner_min, prisoner_max, bed_min, bed_max : int

$ Create domains using the ranges
letting Prisoners be domain int(prisoner_min..prisoner_max),
        Beds be domain int(bed_min..bed_max)

$ Define attribute functions (partial functions, only defined for valid IDs)
given incoming_prisoners_sex, incoming_prisoners_age, incoming_prisoners_supervision: function int --> int
given prison_beds_sex, prison_beds_age, prison_beds_supervision: function int --> int

$ Decision variable: assign each prisoner to a bed (injective ensures one-to-one mapping)
find assignment: function Prisoners --> Beds

such that
$ All prisoners must be assigned (total function)
    forAll prisoner : Prisoners .
        prisoner in defined(assignment),
        
$ Each bed is assigned to at most one prisoner (injective)
    forAll p1, p2 : Prisoners .
        (p1 != p2 /\ p1 in defined(assignment) /\ p2 in defined(assignment)) 
        -> assignment(p1) != assignment(p2),
    
$ Prisoners must be assigned to beds that match their sex
    forAll prisoner : Prisoners .
        prisoner in defined(assignment) ->
        incoming_prisoners_sex(prisoner) = prison_beds_sex(assignment(prisoner)),
    
$ Prisoners must be assigned to beds that match their age category
    forAll prisoner : Prisoners .
        prisoner in defined(assignment) ->
        incoming_prisoners_age(prisoner) = prison_beds_age(assignment(prisoner)),
    
$ Prisoners must be assigned to beds that match their supervision level
    forAll prisoner : Prisoners .
        prisoner in defined(assignment) ->
        incoming_prisoners_supervision(prisoner) = prison_beds_supervision(assignment(prisoner))


```json
{"assignment": {"266": 2, "267": 1, "268": 3, "269": 5, "270": 4, "271": 6, "272": 8, "273": 7, "274": 9, "275": 10, "276": 11, "277": 15, "278": 13, "279": 14, "280": 12}}
```

| Statistic | Value |
|:-|-:|
| SolverTotalTime | 0.02 |
| SATClauses | 36 |
| SavileRowClauseOut | 0 |
| SavileRowTotalTime | 0.322 |
| SolverSatisfiable | 1 |
| SavileRowTimeOut | 0 |
| SATVars | 12 |


If the data is as expected to be fed into the model, we should now have a single set of cell allocations available to us. 

## 2.3 Validate and display the data 
Now that a sollution has been found, lets first check that the data outputted from the constraints model follows the rules of cell allocation - first visually and then programatically. 

In [10]:
def check_all_assignment_match(prisoner, bed):
    return (prisoner['sex'] == bed['sex']) and (prisoner['age_category'] == bed['age_category']) and (prisoner['supervision_level'] == bed['supervision_level'])

valid_selection= True
# Display assignment results with decoded attributes
for i, (prisoner_id, bed_number) in enumerate(assignment.items(), start=1):
    # bed_number is 1-indexed, so we subtract 1 to get the DataFrame row position
    bed_row = cell_spaces.iloc[bed_number - 1]
    
    # Get prisoner details from prisoners_encoded using prisoner_id
    prisoner_details = next((p for p in prisoners_encoded if int(p['prisoner_id']) == int(prisoner_id)), None)
    
    # Decode bed attributes
    bed_decoded = preprocessor.decode_bed_attributes(bed_row)
    
    # Print assignment details
    print(f"[{i}] Prisoner {prisoner_details['name']} (ID: {prisoner_id}) → Bed {bed_row['bed_id']}")
    print(f"    Prisoner: Sex={prisoner_details['sex']}, Age={prisoner_details['age_category']}, Supervision={prisoner_details['supervision_level']}")
    print(f"    Bed: Sex={bed_decoded['sex']}, Age={bed_decoded['age_category']}, Supervision={bed_decoded['supervision_level']}")
    print()
    # Validate allocation matching
    valid_selection = check_all_assignment_match(prisoner_details, bed_decoded)

if valid_selection:
    print("All prisoners have been selected by matching the needed requirements to fufil cell allocation")
else: 
    print("There was a mismatch in cell allocation attributes for some assignments.")









[1] Prisoner Angus "Mad Dog" MacDuff (ID: 266) → Bed 15.0
    Prisoner: Sex=Male, Age=Adult, Supervision=High
    Bed: Sex=Male, Age=Adult, Supervision=High

[2] Prisoner Bruce McLean (ID: 267) → Bed 14.0
    Prisoner: Sex=Male, Age=Adult, Supervision=High
    Bed: Sex=Male, Age=Adult, Supervision=High

[3] Prisoner Ailsa Morrison (ID: 268) → Bed 25.0
    Prisoner: Sex=Female, Age=Adult, Supervision=High
    Bed: Sex=Female, Age=Adult, Supervision=High

[4] Prisoner Callum Strachan (ID: 269) → Bed 47.0
    Prisoner: Sex=Male, Age=Adult, Supervision=Medium
    Bed: Sex=Male, Age=Adult, Supervision=Medium

[5] Prisoner Duncan Forbes (ID: 270) → Bed 46.0
    Prisoner: Sex=Male, Age=Adult, Supervision=Medium
    Bed: Sex=Male, Age=Adult, Supervision=Medium

[6] Prisoner Bonnie MacLeod (ID: 271) → Bed 61.0
    Prisoner: Sex=Female, Age=Adult, Supervision=Medium
    Bed: Sex=Female, Age=Adult, Supervision=Medium

[7] Prisoner Ewan O Harra (ID: 272) → Bed 99.0
    Prisoner: Sex=Male, Age=Adul

As shown above, we can see that the model is able to assign cells successfully given the constraint rules we have given it. This is further affirmed by the programatic check that is done afterwards to ensure that the inbound prisoners requirements match the bed they are assigned to. 